In [1]:
# header
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

import shared_utils

import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) 

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/safety_projects/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# load encampments - these were exported from ArcGIS Pro to get around the 2000 record limit on the open data portal. These are already projected. 
with get_fs().open(f'{GCS_FILE_PATH}Encampments_Projected.geojson') as f:
    encampments = gpd.read_file(f)

In [3]:
# load bridge buffer areas
bridge_areas = gpd.read_parquet(f'{GCS_FILE_PATH}bridgeareas_clean.parquet')

In [4]:
# fixing projection and dropping column - move this over to bridge process script
bridge_areas = bridge_areas.to_crs(3310).drop(["index_right"], axis=1)

In [5]:
# spatial join and aggregate encampments to bridge areas
bridges_encampments = gpd.sjoin(bridge_areas, encampments, how="left")

In [6]:
len(bridges_encampments)

19316

In [7]:
bridges_encampments[bridges_encampments['WONO'].notnull()].head(10)

,geometry,BRIDGE_right,OBJECTID_1_left,OBJECTID_left,DIST_left,CO_left,BRIDGE_X_left,BRIDGE_Y_left,LAT_left,LON_left,NAME_left,LOC_left,YRBLT_left,HST_left,FAC_left,APWID_left,LENG_left,DK_AREA_left,LSW_left,RSW_left,RDW_left,REFVCU_left,VCU_left,MAINSPANS_left,DIR_left,PRINC_left,INTERSEC_left,AADT_left,PCTTRK_left,DEF_left,NHS_left,FUNCTIONAL_left,DATA_EXTRA_left,PM_left,CITY_left,RTE_left,OBJECTID_1_right,OBJECTID_right,DIST_right,CO_right,BRIDGE_X_right,BRIDGE_Y_right,LAT_right,LON_right,NAME_right,LOC_right,YRBLT_right,HST_right,FAC_right,APWID_right,LENG_right,DK_AREA_right,LSW_right,RSW_right,RDW_right,REFVCU_right,VCU_right,MAINSPANS_right,DIR_right,PRINC_right,INTERSEC_right,AADT_right,PCTTRK_right,DEF_right,NHS_right,FUNCTIONAL_right,DATA_EXTRA_right,PM_right,CITY_right,RTE_right,index_right,OBJECTID,FISCAL_YEAR,WO_DATE,WONO,RESP_DIST,ACTIVITY,ACTIVITY_DESCRIPTION,EFIS_REPORTING_CODE,UNITID,FROM_PM,FROM_MILES,XCOORDINATE,YCOORDINATE,WO_COMMENTS,Date,COUNTY,ROUTE
BRIDGE_left,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01 0046,"POLYGON ((-343838.276 435953.537, -343838.276 ...",01 0046,25,25,1,DN,0.766667,1,41.873811,-124.138386,SMITH RIVER OVERFLOW,01-DN-101-35.77,1955,5,U.S. HIGHWAY 101,15.5,86.3,1307,0,0,14.3,N,0,10,2,1,SMITH RIVER OVERFLOW,6700,6,1,1,2,30-Jan-20,35.77,0,101,25,25,1,DN,0.766667,1,41.873811,-124.138386,SMITH RIVER OVERFLOW,01-DN-101-35.77,1955,5,U.S. HIGHWAY 101,15.5,86.3,1307,0,0,14.3,N,0,10,2,1,SMITH RIVER OVERFLOW,6700,6,1,1,2,30-Jan-20,35.77,0,101,4345.0,4346.0,2022,12-MAY-22,6258656.0,01,D42050,UNSHELTERED ENCAMPMENT - CLEANING AND REMOVAL,ENCAMPMENT,01-DN-101,PM 35.75,34.619,-1.381906e+07,5.142053e+06,02 17 2022 Notice to vacate postedCHP Badge 21...,1.652339e+12,DN,101
01 0064,"POLYGON ((-348365.690 424943.910, -348365.690 ...",01 0064,29,29,1,DN,0.000000,0,41.772825,-124.186931,WASHINGTON BLVD OC,01-DN-101-R27.87,1971,5,WASHINGTON BLVD,0.0,73.8,1320,0,0,0.0,H,5,2,2,2,U.S. HIGHWAY 101,29500,10,1,1,2,30-Jan-20,R27.87,0,101,29,29,1,DN,0.000000,0,41.772825,-124.186931,WASHINGTON BLVD OC,01-DN-101-R27.87,1971,5,WASHINGTON BLVD,0.0,73.8,1320,0,0,0.0,H,5,2,2,2,U.S. HIGHWAY 101,29500,10,1,1,2,30-Jan-20,R27.87,0,101,4341.0,4342.0,2022,12-MAY-22,6258447.0,01,D42050,UNSHELTERED ENCAMPMENT - CLEANING AND REMOVAL,ENCAMPMENT,01-DN-101,RPM 27.87,26.797,-1.382451e+07,5.127003e+06,02 17 2022 Notice to vacate postedCHP officer ...,1.652339e+12,DN,101
02 0150L,"POLYGON ((-219012.788 416397.170, -219021.061 ...",02 0150L,135,135,2,SIS,0.000000,0,41.739375,-122.628136,NORTH YREKA SEPARATION,02-SIS-005-R48.23-YRE,1970,5,INTERSTATE 5 NB,11.9,38.0,476,0,0,11.9,H,5,1,1,1,STATE ROUTE 3,8300,25,1,1,11,30-Jan-20,R48.23,YREKA CITY,5,135,135,2,SIS,0.000000,0,41.739375,-122.628136,NORTH YREKA SEPARATION,02-SIS-005-R48.23-YRE,1970,5,INTERSTATE 5 NB,11.9,38.0,476,0,0,11.9,H,5,1,1,1,STATE ROUTE 3,8300,25,1,1,11,30-Jan-20,R48.23,YREKA CITY,5,5116.0,5117.0,2022,14-JUN-22,6300438.0,02,D42050,UNSHELTERED ENCAMPMENT - CLEANING AND REMOVAL,ENCAMPMENT,02-SIS-003,PM 50.09,52.410,-1.365100e+07,5.122100e+06,UNSHELTERED ENCAMPMENT REMOVAL 02 SIS 003 50 090,1.655190e+12,SIS,003
02 0151,"POLYGON ((-218971.826 416648.060, -218971.826 ...",02 0151,137,137,2,SIS,1.850000,2,41.740009,-122.630936,YREKA CREEK,02-SIS-003-L49.99-YRE,1970,5,STATE ROUTE 3,19.5,33.2,668,0,0,19.5,N,0,3,2,1,YREKA CREEK,10100,2,0,1,14,30-Jan-20,L49.99,YREKA CITY,3,137,137,2,SIS,1.850000,2,41.740009,-122.630936,YREKA CREEK,02-SIS-003-L49.99-YRE,1970,5,STATE ROUTE 3,19.5,33.2,668,0,0,19.5,N,0,3,2,1,YREKA CREEK,10100,2,0,1,14,30-Jan-20,L49.99,YREKA CITY,3,5116.0,5117.0,2022,14-JUN-22,6300438.0,02,D42050,UNSHELTERED ENCAMPMENT - CLEANING AND REMOVAL,ENCAMPMENT,02-SIS-003,PM 50.09,52.410,-1.365100e+07,5.122100e+06,UNSHELTERED ENCAMPMENT REMOVAL 02 SIS 003 50 090,1.655190e+12,SIS,003
02 0159L,"POLYGON ((-220417.783 412501.292, -220423.407 ...",02 0159L,147,147,2,SIS,0.000000,0,41.703497,-122.643053,MOONLIT OAKS AVENUE UC,02-SIS-005

In [8]:
# dissolve, grouping by bridge caracteristics, aggregating counts of encampment work orders 
bridges_encampments_agg = bridges_encampments.dissolve(
    by = ["BRIDGE_left","NAME_left","FAC_left"],
    aggfunc = {'WONO' : 'nunique'}
)

In [28]:
# create derived variables: dummy variable, density, ranking
bridges_encampments_agg = bridges_encampments_agg.assign(
    WO_density = bridges_encampments_agg['WONO']/bridges_encampments_agg['geometry'].area,
    WO_dummy = (bridges_encampments_agg['WONO'] > 0).astype(int),
    WO_density_pctile = bridges_encampments_agg['WO_density'].rank(pct=True)
).reset_index()

In [29]:
# save out as geoparquet 
shared_utils.utils.geoparquet_gcs_export(bridges_encampments_agg, GCS_FILE_PATH, "encampments_agg")